In [1]:
import os
import json
#import tqdm
import re
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
#import spacy
import scipy
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from gensim import corpora, models, similarities
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

C:\Users\DELL\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df_ori = pd.read_csv('./data/data job posts.csv')
df_ori.head()
print(df_ori.shape)
df = df_ori.drop_duplicates(['jobpost','Title'])
#df = df[np.isfinite(df['EPS'])]
df = df.dropna(subset=['RequiredQual'])
print(df.shape)
print("Removed {0} duplicates (based on jobpost + Title)".format(df_ori.shape[0]-df.shape[0]))

(19001, 24)
(18458, 24)
Removed 543 duplicates (based on jobpost + Title)


In [3]:
df.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,Location,JobDescription,JobRequirment,RequiredQual,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",AMERIA Investment Consulting Company is seekin...,- Supervises financial management and administ...,"To perform this job successfully, an\r\nindivi...",NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,"IREX Armenia Main Office; Yerevan, Armenia \r\...",NaN,NaN,- Bachelor's Degree; Master's is preferred;\r\...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,"Yerevan, Armenia",Public outreach and strengthening of a growing...,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,"Manila, Philippines",The LEAD (Local Enhancement and Development fo...,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,"Yerevan, Armenia",NaN,- Rendering technical assistance to Database M...,- University degree; economical background is ...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [4]:
df["RequiredQual"] = df['RequiredQual'].astype(str)

In [5]:
df['RequiredQual_token'] = df['RequiredQual'].map(word_tokenize)

In [6]:
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [7]:
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item for item in x if item not in stop_words])
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item for item in x if item not in string.punctuation])

In [8]:
df['RequiredQual_token'] = df['RequiredQual_token'].apply(lambda x: [item.lower() for item in x])

In [9]:
df['RequiredQual_token'].head()

0    [to, perform, job, successfully, individual, m...
1    [bachelor, 's, degree, master, 's, preferred, ...
2    [degree, environmentally, related, field, 5, y...
3    [advanced, degree, public, health, social, sci...
4    [university, degree, economical, background, p...
Name: RequiredQual_token, dtype: object

In [10]:
#Making dictionary out of words
dictionary = corpora.Dictionary(df['RequiredQual_token'])
dictionary.save('/job_req.dict')

In [11]:
 corpus = [dictionary.doc2bow(text) for text in df['RequiredQual_token']]

In [12]:
corpora.MmCorpus.serialize('/job_req.mm', corpus)

In [13]:
corpus = corpora.MmCorpus('/job_req.mm')
print(corpus)

MmCorpus(18458 documents, 14495 features, 825601 non-zero entries)


In [14]:
#define a 2-dimensional LSI space:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

In [15]:
doc = "2 years experience"

In [16]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.5660823326837305), (1, -0.2195380988099088)]


In [17]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [18]:
index.save('/job_req.index')
index = similarities.MatrixSimilarity.load('/job_req.index')

In [19]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims)))

[(0, 0.8066119), (1, 0.7230849), (2, 0.97071385), (3, 0.9675027), (4, 0.96519464), (5, 0.9286672), (6, 0.9081355), (7, 0.9692272), (8, 0.90776646), (9, 0.9038713), (10, 0.7934437), (11, 0.965872), (12, 0.9578781), (13, 0.9424574), (14, 0.9645278), (15, 0.99937767), (16, 0.8591039), (17, 0.97813433), (18, 0.8605493), (19, 0.6854961), (20, 0.93827623), (21, 0.6429313), (22, 0.93827623), (23, 0.91278404), (24, 0.9970962), (25, 0.90837944), (26, 0.946263), (27, 0.8419979), (28, 0.9889207), (29, 0.9709981), (30, 0.94499147), (31, 0.761621), (32, 0.94189227), (33, 0.9693712), (34, 0.99982435), (35, 0.8054752), (36, 0.9765837), (37, 0.88429576), (38, 0.73174417), (39, 0.96554124), (40, 0.9596197), (41, 0.65447366), (42, 0.7474388), (43, 0.9406286), (44, 0.7351957), (45, 0.7398144), (46, 0.8571151), (47, 0.9923601), (48, 0.96017104), (49, 0.9048728), (50, 0.97917557), (51, 0.9917594), (52, 0.83965564), (53, 0.98504543), (54, 0.9807707), (55, 0.9802299), (56, 0.75176233), (57, 0.9955034), (58, 

In [20]:
sims = sorted(list(enumerate(sims)), key=lambda item: -item[1])
top10 = sims[:10]
print(top10)

[(2999, 1.0), (13588, 1.0), (3950, 0.99999994), (13926, 0.99999994), (14284, 0.99999994), (4626, 0.9999999), (4738, 0.9999999), (6579, 0.9999999), (7224, 0.9999999), (11552, 0.9999999)]


In [21]:
pd.set_option('display.max_colwidth', -1)

In [23]:
for index, f in top10:
    #print(index)
    print(df.loc[[index]]["RequiredQual"]+"*********"+ "Similarity: "+str(f))
    #print("***************index*****"+str(index))

2999    - Higher professional education;\r\n- Excellent knowledge of RA Civil, Administrative, Labor and\r\nConstitutional laws;\r\n- Knowledge of International Law is desired;\r\n- At least 2 years of professional experience;\r\n- Excellent legal writing and presentation skills;\r\n- Excellent knowledge of Armenian and Russian languages. Good knowledge\r\nof English is desired;\r\n- Knowledge of computer is desired.*********Similarity: 1.0
Name: RequiredQual, dtype: object
13588    - University degree in Economics, Marketing or Advertising;\r\n- At least 2 years of experience in a relevant field;\r\n- Experience in advertising agency is an asset;\r\n- Knowledge and understanding of all processes related to the planning,\r\nbudgeting, development and production of ATL  BTL campaigns/ and\r\ncommunication materials;\r\n- Reporting & business writing skills;\r\n- Innovative and analytical thinking;\r\n- Negotiation skills;\r\n- Time management skills;\r\n- Ability to work in a team;\r\n-